# Cross validation of the potential fit
The functions required to run the fit in a Jupyter notebook are imported from the source code, along with matplotlib, and glob.

In [ ]:
import lammps_potenial_fitting.fitting_output as fit_out
import lammps_potenial_fitting.cross_validation as cv
import matplotlib.pyplot as plt
import glob

## Setup of fitting parameters for your structure (Example: core-shell LiNiO$_2$)
Params is the dictionary (of dictionaries) which contains the main information relating to the system and potentials. There are 5 sub dictionaries: core_shell, charges, masses, potentials, and cs_springs.

**core_shell**: The keys are each atom type within the structure, with the relating value a boolean expression stating if that atom type is core-shell or not i.e. True = core-shell, False = rigid ion.

**charges**: The keys are again each atom type within the structure. The relating value is either a float representation of the atomic charge (for rigid ion atoms) or a sub dictionary where the sub keys are 'core' and 'shell', with the relating sub values being a float representation of the charge. Note: if you are fitting the charge separation (dq), the formal charge should be on the core and 0.0 charge on the shell.

**masses**: Similar to charges, the keys are each atom type within the structure, with the values either a float representation of the atomic mass, or a sub directory with the sub keys 'core' and 'shell' and sub values a float representation of the mass on each (summing to the atomic mass). Mass can not be fitted, and there is no definative way of splitting the mass, however convention suggests having 10 % mass on the shell.

**potentials**: The keys are atom label pairs separated by a dash (str), example: `Li-O`. The values are a list of the buckingham potential parameters, i.e. `[a, rho, c]`, where each parameter is a float.

**cs_springs**: The keys are agian atom label pairs separated by a dash (str), example: `O-O`. This basically denotes the spring is between 'O' core and 'O' shell. The values are a list of the spring contants, k1 and k2, as floats. Commonly k2 is set to 0.0.

**NOTE: `masses` AND `core_shell` SHOULD BE THE SAME AS THE PARAMETERS DURING THE FIT.**

In [ ]:
params = {}
params['core_shell'] = { 'Li': False, 'Ni': False, 'O': True }
params['charges'] = {'Li': +1.0,
                     'Ni': +3.0,
                     'O': {'core':  -2.0,
                           'shell': 0.0}} 
params['masses'] = {'Li': 6.941,
                    'Ni': 58.6934,
                    'O': {'core': 14.3991,
                          'shell': 1.5999} }
params['potentials'] = {'Li-O': [663.111, 0.119, 0.0],
                        'Ni-O': [1393.540, 0.218, 0.000],
                        'O-O': [25804.807, 0.284, 0.0]}
params['cs_springs'] = {'O-O' : [20.0, 0.0]}

## Define directory paths and names/number of structures

Structures, structures_to_fit, and fits are required inputs. These designate how many structures are in the training set (saved in the directory named `vaspruns`), how many of those structures to fit to (to match with cross-validation), and how many cross-validations to conduct, respectively. **NB: Carefully consider the last point. You can not cross-validate with structures in the fit, therefore if you have fitted to 10/15 structures, you can not validate to 10 structures as only 5 are avaliable.**

The `head_directory_name` is set up to output the fit to a results directory, with a sub directory of the number of structures fitted, i.e. where your data is saved for each fit of that type. For example, if you have fit 5 structures, your data would likely be in 'results/5_fit'. This can be different if you have changed the default location. `cv_directory_name` is the name of the sub directory where you wish to save your cross-validation data. The combination of these directories makes the output directory `head_output_directory`.

In [ ]:
structures = 15 #Total number of structures in the training set
structures_in_fit = 5 #Number of structures you wish to fit to
fits = 8 #Number of fits to run

# Create cross validation directory
head_directory_name = 'results/{}_fit'.format(structures_in_fit)
cv_directory_name = 'cross_validation'
head_output_directory = fit_out.create_directory(head_directory_name, cv_directory_name)

## Runs cross-validation using the fitted parameters with other structures in training set

The cross-validation itself is run within the function called `run_cross_validation` in `cross_validation.py`, which is executed here. There are 2 required inputs: `head_directory_name` and `params` which are defined above. There is also 1 optional input, `supercell` which creates a supercell of the structures before running the fit. This can either be a list with the multipliers for x, y, and z, i.e. [2,2,2], or a list of list if you want to use different system sizes (not recommended) or have different starting sizes, i.e. [[2,2,2],[1,1,2]]. **Note: you want the cell to be the same size as that used in the fit for direct comparison.**


Output data is sent to the `head directory` location, in a sub directory named `cross_validation`. Each cross-validation set is saved in a sub directory named with the structure numbers used, prefixed with `p` denoting the potential was fitted to those structures. Inside the directory, the files are prefixed with `s` denoting the structures the potential was validated with, followed with the structure numbers in the validation set and the suffix stating what the file contains i.e. `dft_forces`.

In [ ]:
cv.run_cross_validation(fits, structures, structures_in_fit, head_directory_name, head_output_directory, params, supercell=[2,2,2], seed=False)

## Plotting the cross-validation $\chi^{2}$ errors

Firstly, for each cross-validation set, the errors are read in from the sub directories within the `head_output_directory` and stored in a dictionary, converting the error to a float and using the sub directory names as the structure numbers (x-axis labels) by removing the leading head directory path and error file extention. This won't work for a directory tree, only for a depth of 1.

The cross-validation errors are then plotted and saved in the output directory, i.e. the `cross_validation` directory. There are options to change the title and degree of rotaion on the x-axis labels. You can also chose whether to save the plot or not. Further editing and formatting can be done by changing the `plot_cross_validation` function in `plotting.py`.

In [ ]:
for cv_directory in sorted(glob.glob('{}/*'.format(head_output_directory))):
    if '.png' in cv_directory:
        continue
    error_dict = cv.setup_error_dict(cv_directory)
    cv.plot_cross_validation(error_dict, cv_directory, head_output_directory, xlabel_rotation=50, title='default', save=True)